# package_LAB

In [2]:
# Correct import statements and package reloads
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from matplotlib import colors as mcolors
from ipywidgets import interact, FloatSlider,Layout
import package_DBR

import package_LAB

from importlib import reload
package_LAB = reload(package_LAB)
from package_DBR import myRound, SelectPath_RT, Delay_RT,Process,Bode
from package_LAB import LL_RT,PID_RT, IMCTuning, PID



ImportError: cannot import name 'PID' from 'package_LAB' (C:\Users\Bastien\Downloads\control_theory_software_v3\Control_theory_software_V3\Labo_Control_Theory\package_LAB.py)

# LL_RT()

In [ ]:
help(LL_RT)

In [ ]:
# Simulation parameters
TSim = 100
Ts = 1
N = int(TSim/Ts) + 1

# Path for MV
MVPath = {0: 0, 5: 1, 50: 2, 80: 3, TSim: 3}

# Initializing vectors
t = []
MV = []
MVDelay = []
PV_EBD = []
PV_EFD = []
PV_TRAP = []

# Parameters
Kp = 1
Tla = 10
Tle = 15
T = 5
theta = 10

# Main loop
for i in range(0, N):
    t.append(i*Ts)
    SelectPath_RT(MVPath, t, MV)
    Delay_RT(MV, theta, Ts, MVDelay)
    LL_RT(MVDelay, Kp, Ts, Tle, Tla, PV_EBD, 0, 'EBD')
    LL_RT(MVDelay, Kp, Ts, Tle, Tla, PV_EFD, 0, 'EFD')
    # LL_RT(MVDelay, Kp, Ts, Tle, Tla, T, PV_TRAP, 0, 'TRAP')

# Plotting results
plt.figure(figsize=(15, 9))

plt.subplot(2, 1, 1)
plt.step(t, MV, 'b-', label='MV', where='post')
plt.step(t, MVDelay, 'b-', linewidth=0.5, label='MV plus delay', where='post')
plt.ylabel('Value of MV')
plt.title('First order plus delay response')
plt.legend(loc='best')
plt.xlim([0, TSim])

plt.subplot(2, 1, 2)
plt.step(t, PV_EBD, 'g-', label='First order response plus delay (EBD)', where='post')
plt.step(t, PV_EFD, 'lime', label='First order response plus delay (EFD)', where='post')
# plt.step(t, PV_TRAP, 'springgreen', label='First order response plus delay (TRAP)', where='post')
plt.ylabel('Value of PV')
plt.xlabel('Time [s]')
plt.legend(loc='best')
plt.xlim([0, TSim])

# PID_RT

In [ ]:
help(PID_RT)

In [ ]:
# Initialize
t = []
TSim = 1200
Ts = 0.01
N = int(TSim/Ts) + 1
PVInit = 0
MVmin = 0
MVmax = 100
ManStart = 600
ManEnd = 700

SP = []
MV = []
PV = []
DV =[]
Man = []
MVMan = []
MVFF = []
MVP = []
MVI = []
MVD = [] 
E  = []

# Paths
MVManPath = {0: 50, TSim: 50}  #Valeur MV en mode manuel jusqu'à valeur max de temps TSim
SPPath = {0: 50, 5: 60, TSim: 60}
PVPath = {0: 50, 5: 50, TSim: 50}
ManPath = {0: False, ManStart: True, ManEnd: False, TSim: False} #Timing du passage en mode manuel
MVFFPath = {0: 0, 1100 :60, TSim: 60}

# Parameters
Kc = 1
Ti = 50
Td = 5
alpha = 0.6

def update_plot(Kc, Ti, Td, alpha, TSim, ManStart,ManEnd):
    t.clear()
    SP.clear()
    PV.clear()
    Man.clear()
    MVMan.clear()
    MVFF.clear()
    MV.clear()
    MVP.clear()
    MVI.clear()
    MVD.clear()
    E.clear()
    # Update ManPath with new ManStart and ManEnd values from sliders
    ManPath = {0: False, ManStart: True, ManEnd: False, TSim: False}
    
    for i in range(0, N):
        t.append(i * Ts)
        SelectPath_RT(SPPath, t, SP)
        SelectPath_RT(PVPath, t, PV)
        SelectPath_RT(ManPath, t, Man)
        SelectPath_RT(MVManPath, t, MVMan)
        SelectPath_RT(MVFFPath, t, MVFF)
        
        PID_RT(SP, PV, Man, MVMan, MVFF, Kc, Ti, Td, alpha, Ts, MVmin, MVmax, MV, MVP, MVI, MVD, E, True, PVInit, "EBD-EBD")
    
    
    # Plot MV  
    plt.figure(figsize=(15, 15))
    plt.subplot(4, 1, 1)
    plt.step(t, MV, 'b-', label='MV', where='post')
    plt.step(t, MVP, '--y', label='MVP', where='post')
    plt.step(t, MVI, '--c', label='MVI', where='post')
    plt.step(t, MVD, '--r', label='MVD', where='post')
    plt.step(t, MVFF, '--g', label='MVFF', where='post')
    plt.step(t, E, 'k-', linewidth=1, label='E', where='post')
    plt.ylabel('Value of MV')
    plt.title('PID response')
    plt.legend(loc='best')
    plt.xlim([0, TSim])
    
    # Plot Man
    plt.subplot(4, 1, 2)
    plt.step(t, Man, 'b-', linewidth=1, label='Man', where='post')
    plt.ylabel('Value of Man')
    plt.legend(loc='best')
    plt.xlim([0, TSim])
    
#sliders
Kc_slider = FloatSlider(value=Kc, min=0.1, max=5.0, step=0.1, description='Kc:', layout=Layout(width='50%'))
Ti_slider = FloatSlider(value=Ti, min=0.1, max=100.0, step=0.1, description='Ti:', layout=Layout(width='50%'))
Td_slider = FloatSlider(value=Td, min=1, max=20, step=1, description='Td:', layout=Layout(width='50%'))
alpha_slider = FloatSlider(value=alpha, min=0.1, max=1.0, step=0.1, description='Alpha:', layout=Layout(width='50%'))
ManStart_slider = FloatSlider(value=ManStart, min=100, max=2000, step=100, description='ManStart:', layout=Layout(width='50%'))
ManEnd_slider = FloatSlider(value=ManEnd, min=100, max=2000, step=100, description='ManEnd:', layout=Layout(width='50%'))
TSim_slider = FloatSlider(value=TSim, min=100, max=2000, step=100, description='TSim:', layout=Layout(width='50%'))

interact(update_plot, Kc=Kc_slider, Ti=Ti_slider, Td=Td_slider, alpha=alpha_slider, TSim = TSim_slider, ManStart = ManStart_slider, ManEnd = ManEnd_slider)


# IMCTuning()

In [ ]:
help(IMCTuning)

In [ ]:
# Parameters for input-output dynamic
Kp = 0.48
T1p = 160
T2p = 11
thetap = 4

# Parameters PID controller
gamma = 0.6                                # A faire : Discuter variation de gamma
Kc, Ti, Td = IMCTuning(Kp,T1p,T2p,thetap,gamma, 'SOPDT')

print("Kc = ",Kc)
print("Ti = ",Ti)
print("Td = ",Td)

# StabilityMargins()